# Strands Travel Agent を Amazon EKS にデプロイし、AgentCore Observability と Evaluations を使用

このノートブックは、[Strands Agents SDK](https://github.com/strands-agents/sdk-python) で構築された Travel Agent を Amazon EKS にデプロイするプロセスを自動化します。

## 前提条件

- [AWS CLI](https://aws.amazon.com/cli/) がインストールされ設定されていること
- [eksctl](https://eksctl.io/installation/) (v0.208.x 以降) がインストールされていること
- [Helm](https://helm.sh/) (v3 以降) がインストールされていること
- [kubectl](https://docs.aws.amazon.com/eks/latest/userguide/install-kubectl.html) がインストールされていること
- [Docker](https://www.docker.com/) がインストールされ実行中であること
- AWS アカウントで Amazon Bedrock Claude モデルが有効であること

In [ ]:
# macOS で前提条件をインストールするにはコメントを解除して実行
# !brew tap weaveworks/tap
# !brew install weaveworks/tap/eksctl
# !brew install helm
# !brew install kubectl

# インストールの確認
!echo "=== Checking installed versions ==="
!aws --version
!eksctl version
!helm version --short
!kubectl version --client
!docker --version

## 1. 設定

デプロイ用の環境変数を設定します。必要に応じてこれらの値を変更してください。

In [ ]:
import os

# AWS アカウント ID を自動検出
account_id = !aws sts get-caller-identity --query 'Account' --output text
os.environ['AWS_ACCOUNT_ID'] = account_id[0]

# ユーザー設定可能な設定（環境から読み込むか、デフォルトを使用）
# デプロイをカスタマイズするには、実行前にこれらの環境変数を設定
os.environ['AWS_REGION'] = os.getenv('AWS_REGION', 'us-east-1')
os.environ['CLUSTER_NAME'] = os.getenv('CLUSTER_NAME', 'eks-strands-agents-demo')
os.environ['SERVICE_NAME'] = os.getenv('SERVICE_NAME', 'strands-agents-travel')

# CloudWatch 設定
os.environ['LOG_GROUP_NAME'] = os.getenv('LOG_GROUP_NAME', '/strands-agents/travel')
os.environ['LOG_STREAM_NAME'] = os.getenv('LOG_STREAM_NAME', 'agent-logs')
os.environ['METRIC_NAMESPACE'] = os.getenv('METRIC_NAMESPACE', 'StrandsAgents/Travel')

# ポート設定
os.environ['LOCAL_PORT'] = os.getenv('LOCAL_PORT', '8080')
os.environ['SERVICE_PORT'] = os.getenv('SERVICE_PORT', '80')

# 設定を表示
print("=== Deployment Configuration ===")
print(f"AWS Account ID: {os.environ['AWS_ACCOUNT_ID']}")
print(f"AWS Region: {os.environ['AWS_REGION']}")
print(f"Cluster Name: {os.environ['CLUSTER_NAME']}")
print(f"Service Name: {os.environ['SERVICE_NAME']}")
print(f"Log Group: {os.environ['LOG_GROUP_NAME']}")
print(f"Log Stream: {os.environ['LOG_STREAM_NAME']}")
print(f"Metric Namespace: {os.environ['METRIC_NAMESPACE']}")
print(f"Local Port: {os.environ['LOCAL_PORT']}")
print(f"Service Port: {os.environ['SERVICE_PORT']}")

## 2. CloudWatch ロググループの作成

OpenTelemetry ログ用の CloudWatch ロググループとストリームを作成します。

In [ ]:
%%bash
echo "Creating CloudWatch log group and stream..."
aws logs create-log-group --log-group-name ${LOG_GROUP_NAME} --region ${AWS_REGION} 2>/dev/null || echo "Log group already exists"
aws logs create-log-stream --log-group-name ${LOG_GROUP_NAME} --log-stream-name ${LOG_STREAM_NAME} --region ${AWS_REGION} 2>/dev/null || echo "Log stream already exists"
echo "CloudWatch resources ready!"

## 3. Dockerfile を CloudWatch 設定で更新

Dockerfile のプレースホルダー値を実際の CloudWatch 設定に置き換えます。

In [ ]:
import os
import shutil

dockerfile_path = 'docker/Dockerfile'
dockerfile_backup = 'docker/Dockerfile.backup'

# ファイルが存在することを確認
if not os.path.exists(dockerfile_path):
    raise FileNotFoundError(f"Dockerfile not found at {dockerfile_path}. Make sure you're running from the correct directory.")

# Dockerfile を読み込み
with open(dockerfile_path, 'r') as f:
    content = f.read()

# プレースホルダーが存在するか確認
if '<YOUR_LOG_GROUP>' in content or '<YOUR_SERVICE_NAME>' in content:
    # 変更前にバックアップを作成
    shutil.copy(dockerfile_path, dockerfile_backup)
    print(f"Backup created: {dockerfile_backup}")
    
    # プレースホルダーを実際の値に置換
    content = content.replace('<YOUR_SERVICE_NAME>', os.environ['SERVICE_NAME'])
    content = content.replace('<YOUR_LOG_GROUP>', os.environ['LOG_GROUP_NAME'])
    content = content.replace('<YOUR_LOG_STREAM>', os.environ['LOG_STREAM_NAME'])
    content = content.replace('<YOUR_METRIC_NAMESPACE>', os.environ['METRIC_NAMESPACE'])
    
    # 書き戻し
    with open(dockerfile_path, 'w') as f:
        f.write(content)
    print("Dockerfile updated with configuration values")
else:
    print("Dockerfile already configured (no placeholders found)")
    print("To restore placeholders, copy Dockerfile.backup to Dockerfile")

# 更新された OTEL 行を表示
print("\\nCurrent OTEL configuration:")
for line in content.split('\\n'):
    if 'OTEL_RESOURCE_ATTRIBUTES' in line or 'OTEL_EXPORTER_OTLP_LOGS_HEADERS' in line:
        print(f"  {line}")

## 4. EKS クラスターの作成

EKS Auto Mode クラスターを作成します。このステップには約 15-20 分かかります。

In [ ]:
%%bash
echo "Creating EKS Auto Mode cluster: $CLUSTER_NAME"
echo "This will take approximately 15-20 minutes..."

eksctl create cluster --name $CLUSTER_NAME --region $AWS_REGION --enable-auto-mode

In [ ]:
%%bash
# kubeconfig コンテキストを設定
aws eks update-kubeconfig --name $CLUSTER_NAME --region $AWS_REGION

# クラスターアクセスを確認
echo "Cluster nodes:"
kubectl get nodes

## 5. Docker イメージをビルドして ECR にプッシュ

Travel Agent の Docker イメージをビルドして Amazon ECR にプッシュします。

`重要な注意` - ローカルで Docker インスタンスが実行されていることを確認してください

In [ ]:
%%bash
# Amazon ECR に認証
echo "Authenticating to ECR..."
aws ecr get-login-password --region ${AWS_REGION} | docker login --username AWS --password-stdin ${AWS_ACCOUNT_ID}.dkr.ecr.${AWS_REGION}.amazonaws.com

# ECR リポジトリを作成（既に存在する場合はエラーを無視）
echo "Creating ECR repository..."
aws ecr create-repository --repository-name ${SERVICE_NAME} --region ${AWS_REGION} 2>/dev/null || echo "Repository already exists"

In [ ]:
%%bash
# Docker イメージをビルド
echo "Building Docker image..."
docker build --platform linux/amd64 -t ${SERVICE_NAME}:latest docker/

# ECR 用にイメージをタグ付け
docker tag ${SERVICE_NAME}:latest ${AWS_ACCOUNT_ID}.dkr.ecr.${AWS_REGION}.amazonaws.com/${SERVICE_NAME}:latest

# ECR にイメージをプッシュ
echo "Pushing image to ECR..."
docker push ${AWS_ACCOUNT_ID}.dkr.ecr.${AWS_REGION}.amazonaws.com/${SERVICE_NAME}:latest

echo "Docker image pushed successfully!"

## 6. IAM ポリシーの設定

Amazon Bedrock と CloudWatch Logs の権限を持つ IAM ポリシーを作成します。

In [ ]:
%%bash
# Bedrock と CloudWatch Logs の権限を持つ IAM ポリシーを作成
cat > /tmp/travel-agent-policy.json << 'EOF'
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "bedrock:InvokeModel",
        "bedrock:InvokeModelWithResponseStream"
      ],
      "Resource": "*"
    },
    {
      "Effect": "Allow",
      "Action": [
        "logs:CreateLogGroup",
        "logs:CreateLogStream",
        "logs:PutLogEvents",
        "logs:DescribeLogGroups",
        "logs:DescribeLogStreams"
      ],
      "Resource": "*"
    }
  ]
}
EOF

# IAM ポリシーを作成（既に存在する場合はエラーを無視）
aws iam create-policy \
  --policy-name ${SERVICE_NAME}-policy \
  --policy-document file:///tmp/travel-agent-policy.json 2>/dev/null || echo "Policy already exists"

rm -f /tmp/travel-agent-policy.json
echo "IAM policy ready!"

## 7. EKS Pod Identity の作成

サービスアカウント用の EKS Pod Identity 関連付けを作成します。

In [ ]:
%%bash
# EKS Pod Identity 関連付けを作成
echo "Creating Pod Identity association..."
eksctl create podidentityassociation --cluster $CLUSTER_NAME \
  --namespace default \
  --service-account-name ${SERVICE_NAME} \
  --permission-policy-arns arn:aws:iam::${AWS_ACCOUNT_ID}:policy/${SERVICE_NAME}-policy \
  --role-name eks-${SERVICE_NAME} \
  --region $AWS_REGION

echo "Pod Identity association created!"

## 8. CloudWatch Observability アドオンのインストール（オプション）

> **注意:** このステップは**オプション**です。CloudWatch Observability アドオンは Bedrock AgentCore Observability には**必要ありません**。AgentCore は Dockerfile の OTEL 設定を使用してテレメトリを直接 CloudWatch に送信します。AgentCore 可観測性のみが必要な場合は、このセクションをスキップしてください。

CloudWatch Observability アドオンをインストールして、（AgentCore テレメトリ以外の）追加の Kubernetes レベルのメトリクスとログを収集します。

In [ ]:
%%bash
# CloudWatch エージェント用の Pod Identity を作成
echo "Creating CloudWatch agent Pod Identity..."
eksctl create podidentityassociation --cluster $CLUSTER_NAME \
  --namespace amazon-cloudwatch \
  --service-account-name cloudwatch-agent \
  --permission-policy-arns arn:aws:iam::aws:policy/CloudWatchAgentServerPolicy \
  --role-name eks-cloudwatch-agent \
  --region $AWS_REGION

echo "CloudWatch agent Pod Identity created!"

In [ ]:
%%bash
# CloudWatch Observability アドオンをインストール
echo "Installing CloudWatch Observability addon..."
aws eks create-addon \
  --addon-name amazon-cloudwatch-observability \
  --cluster-name $CLUSTER_NAME \
  --region $AWS_REGION

echo "Waiting for addon to be active..."
aws eks wait addon-active --cluster-name $CLUSTER_NAME --addon-name amazon-cloudwatch-observability --region $AWS_REGION
echo "CloudWatch Observability addon installed!"

## 9. Helm Chart のデプロイ

Helm chart を使用して Travel Agent アプリケーションをデプロイします。

In [ ]:
%%bash
# chart ディレクトリが存在することを確認
if [ ! -d "./chart" ]; then
    echo "Error: Helm chart directory './chart' not found"
    echo "Make sure you're running from the strands-travel-agent-eks directory"
    exit 1
fi

# Helm でデプロイ（冪等性のために upgrade --install を使用）
echo "Deploying Travel Agent with Helm..."
helm upgrade --install ${SERVICE_NAME} ./chart \
  --set image.repository=${AWS_ACCOUNT_ID}.dkr.ecr.${AWS_REGION}.amazonaws.com/${SERVICE_NAME} \
  --set image.tag=latest

echo "Helm deployment initiated!"

In [ ]:
%%bash
# デプロイが利用可能になるまで待機
echo "Waiting for deployment to be ready..."
kubectl wait --for=condition=available deployments ${SERVICE_NAME} --timeout=300s

# Pod ステータスを確認
echo "\\nPod status:"
kubectl get pods -l app.kubernetes.io/name=${SERVICE_NAME}

## 10. ポートフォワードの開始

バックグラウンドでポートフォワーディングを開始して、ローカルで Travel Agent にアクセスできるようにします。

In [ ]:
import subprocess
import time
import os

local_port = os.environ.get('LOCAL_PORT', '8080')
service_port = os.environ.get('SERVICE_PORT', '80')
service_name = os.environ['SERVICE_NAME']

# バックグラウンドでポートフォワードを開始
port_forward = subprocess.Popen(
    ["kubectl", "--namespace", "default", "port-forward", f"service/{service_name}", f"{local_port}:{service_port}"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)
print(f"Port-forward started (PID: {port_forward.pid})")
print(f"Agent will be available at: http://localhost:{local_port}/travel")
print("\\nNote: Run the 'Stop Port Forward' cell below when done testing")
time.sleep(5)  # ポートフォワードが確立されるまで待機

## 11. エージェントのテスト

テストクエリで Travel Agent を呼び出します。

In [ ]:
import requests
import os

local_port = os.environ.get('LOCAL_PORT', '8080')
url = f"http://localhost:{local_port}/travel"
payload = {"prompt": "What are the best places to visit in Tokyo in March?"}

print(f"Sending request to: {url}")
print(f"Prompt: {payload['prompt']}")
print("\\nWaiting for response (this may take a minute)...\\n")

try:
    response = requests.post(url, json=payload, timeout=120)
    print(f"Status: {response.status_code}")
    print(f"\\nResponse:\\n{response.text}")
except requests.exceptions.ConnectionError as e:
    print(f"Connection failed: {e}")
    print("\\nMake sure port-forward is running (run the cell above)")

## 12. ポートフォワードの停止

テストが完了したらポートフォワードプロセスを停止します。

In [ ]:
# ポートフォワードプロセスを停止
if 'port_forward' in dir() and port_forward.poll() is None:
    port_forward.terminate()
    print("Port-forward stopped")
else:
    print("Port-forward not running")

## 13. クリーンアップ（オプション）

このノートブックで作成されたすべてのリソースを削除するには、以下のセルのコメントを解除して実行してください。

In [ ]:
# %%bash
# # helm chart をアンインストール
# echo "Uninstalling helm chart..."
# helm uninstall ${SERVICE_NAME}

In [ ]:
# %%bash
# # EKS クラスターを削除（数分かかります）
# echo "Deleting EKS cluster: $CLUSTER_NAME"
# echo "This will take several minutes..."
# eksctl delete cluster --name $CLUSTER_NAME --region $AWS_REGION --wait

In [ ]:
# %%bash
# # IAM ポリシーを削除
# echo "Deleting IAM policy..."
# aws iam delete-policy --policy-arn arn:aws:iam::${AWS_ACCOUNT_ID}:policy/${SERVICE_NAME}-policy

In [ ]:
# %%bash
# # ECR リポジトリを削除
# echo "Deleting ECR repository..."
# aws ecr delete-repository --repository-name ${SERVICE_NAME} --region ${AWS_REGION} --force

In [ ]:
# %%bash
# # CloudWatch ロググループを削除
# echo "Deleting CloudWatch log group..."
# aws logs delete-log-group --log-group-name ${LOG_GROUP_NAME} --region ${AWS_REGION}